In [4]:
import os
import openai




azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import requests

# Initialiser le modèle ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                         engine=deployment_name)

C:\Users\antoa\AppData\Local\Temp\ipykernel_16588\2979787027.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo",
WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [6]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS 

# Load documents from the CSV
loader = CSVLoader(file_path="gautenberg.csv", encoding="utf-8")
documents = loader.load()

# Initialize the Hugging Face embeddings model
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 

# Create the FAISS vector store from the loaded documents
vectorstore = FAISS.from_documents(documents, embeddings_model)

C:\Users\antoa\AppData\Local\Temp\ipykernel_16588\3519297463.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\antoa\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\antoa\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenizatio

save 
vectorstore.save_local("vectorestore")

In [7]:
vectorestore = FAISS.load_local("vectorestore", embeddings_model, allow_dangerous_deserialization=True)

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import requests
from langchain_openai import ChatOpenAI

# Initialiser le modèle ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                        openai_api_base=azure_openai_api_endpoint
                         )



In [9]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


In [10]:
import pandas as pd
df = pd.read_csv("gautenberg.csv", encoding="utf-8")

# Créer un objet RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorestore.as_retriever())

# Fonction pour poser une question à propos d'un livre
def ask_book_question(question):
    response = qa_chain({"query": question})
    return response["result"]

# Fonction pour obtenir les informations d'un livre
def get_book_info(book_title):
    # Filtrer le DataFrame pour trouver les informations du livre par titre
    book_info = df[df['Title'].str.contains(book_title, case=False, na=False)]
    if not book_info.empty:
        # Récupérer les attributs principaux du livre
        book_attributes = [
            f"Titre: {book_info.iloc[0]['Title']}",
            f"Auteur: {book_info.iloc[0]['Author']}",
            f"Langue: {book_info.iloc[0]['Language']}",
            f"Sujet: {book_info.iloc[0]['Subject']}",
            f"Résumé: {book_info.iloc[0]['Summary']}",
        ]
        return book_attributes, book_info.iloc[0]['Title'], book_info.iloc[0]['Ebook ID']
    else:
        # Retourner None si le livre n'est pas trouvé
        return None, None, None




In [ ]:
# Fonction pour poser une question à propos d'un livre et obtenir une réponse synthétique
def ask_and_generate_response(book_title, question):
    book_info, title, ebook_id = get_book_info(book_title)
    if book_info is not None:
        # Ajouter le titre du livre à la question pour la clarté
        question_prompt = f"Pour le livre '{title}', {question}"
        response = ask_book_question(question_prompt)
        return response
    else:
        return "Désolé, je n'ai pas trouvé d'informations sur ce livre."

# Exemple d'utilisation
book_title = "Pride and Prejudice"  # Titre du livre à interroger
question = "Quel est le thème principal de ce livre ?"  # Question à poser
response = ask_and_generate_response(book_title, question)
print(response)

In [11]:


# Créer un objet RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorestore.as_retriever())

# Fonction pour poser une question à propos d'un livre
def ask_book_question(question):
    response = qa_chain({"query": question})
    return response["result"]

# Fonction pour récupérer le texte d'un livre à partir de Gutenberg
def fetch_book_text(ebook_id):
    url = f"https://www.gutenberg.org/ebooks/{ebook_id}.txt.utf-8"  # URL pour le téléchargement en .txt
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Demander à l'utilisateur de saisir le titre du livre
book_title = input("Veuillez entrer le titre du livre : ")

# Poser une question sur le livre en utilisant l'embedding
question = f"Quels sont les attributs du livre intitulé '{book_title}' ?"
attributes_response = ask_book_question(question)
print(f"Attributs du livre : {attributes_response}")

# Demander à l'utilisateur s'il souhaite récupérer le texte du livre
fetch_text = input("Souhaitez-vous récupérer le texte complet du livre ? (oui/non) : ")

if fetch_text.lower() == 'oui':
    ebook_id = attributes_response.get('Ebook ID', None)  # Vous devrez extraire l'ebook_id de la réponse si nécessaire
    if ebook_id:
        book_text = fetch_book_text(ebook_id)
        if book_text:
            print("Texte du livre récupéré avec succès.")
            # Poser une question sur le texte du livre
            question_text = input("Posez votre question sur le texte du livre : ")
            answer_text = ask_book_question(question_text)
            print(f"Réponse : {answer_text}")
        else:
            print("Erreur lors de la récupération du texte du livre.")
else:
    print("Aucun livre trouvé avec ce titre.")

C:\Users\antoa\AppData\Local\Temp\ipykernel_16588\4027767616.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": question})


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}